In [152]:
import pandas as pd
import numpy as np

In [160]:
cols_to_drop = ['anger', 'joy', 'optimism', 'sadness', 'not-hate', 'hate', 'non_irony', 'not-offensive', 'offensive',
               'negative', 'neutral', 'positive', 'label']

In [161]:
df_train.columns

Index(['anger', 'joy', 'optimism', 'sadness', 'not-hate', 'hate', 'non_irony',
       'not-offensive', 'offensive', 'negative', 'neutral', 'positive'],
      dtype='object')

In [162]:
df_train = pd.read_csv('dataset_preds_all_models_train.csv')[cols_to_drop]#.drop(columns=cols_to_drop)
df_train.head()

,anger,joy,optimism,sadness,not-hate,hate,non_irony,not-offensive,offensive,negative,neutral,positive,label
0,0.115456,0.011387,0.195406,0.677751,0.983329,0.016671,0.849077,0.757905,0.242095,0.438617,0.422358,0.139025,happy
1,0.966903,0.006743,0.005363,0.020991,0.909270,0.090730,0.921817,0.665683,0.334317,0.919669,0.073009,0.007322,angry
2,0.009321,0.957922,0.016562,0.016195,0.972480,0.027520,0.905163,0.978436,0.021564,0.001311,0.006323,0.992365,happy
3,0.008275,0.959311,0.019986,0.012427,0.933811,0.066188,0.948829,0.909642,0.090358,0.001438,0.029523,0.969039,happy
4,0.976041,0.004920,0.011711,0.007327,0.793186,0.206814,0.432330,0.419770,0.580230,0.909924,0.083572,0.006504,disappointed


In [163]:
df_test = pd.read_csv('dataset_preds_all_models_test.csv')[cols_to_drop]#.drop(columns=cols_to_drop)
df_test.head()

,anger,joy,optimism,sadness,not-hate,hate,non_irony,not-offensive,offensive,negative,neutral,positive,label
0,0.008511,0.965951,0.013212,0.012326,0.974149,0.025850,0.849865,0.977916,0.022084,0.001440,0.006966,0.991595,happy
1,0.899331,0.053033,0.036412,0.011225,0.774592,0.225408,0.795617,0.874663,0.125337,0.239116,0.604198,0.156686,angry
2,0.282376,0.043560,0.626704,0.047360,0.927542,0.072458,0.968107,0.816657,0.183343,0.167623,0.488834,0.343542,happy
3,0.011135,0.952730,0.022653,0.013482,0.959886,0.040114,0.952556,0.952512,0.047487,0.000974,0.017529,0.981497,happy
4,0.007975,0.968310,0.012793,0.010921,0.970835,0.029165,0.952927,0.974449,0.025551,0.001060,0.007141,0.991798,happy


In [164]:
df_test.columns

Index(['anger', 'joy', 'optimism', 'sadness', 'not-hate', 'hate', 'non_irony',
       'not-offensive', 'offensive', 'negative', 'neutral', 'positive',
       'label'],
      dtype='object')

In [165]:
print(df_train.shape, df_test.shape)

(294, 13) (74, 13)


In [166]:
X_train = df_train.drop(columns='label')
y_train = df_train['label']
print(X_train.shape, y_train.shape, df_train.shape)

(294, 12) (294,) (294, 13)


In [167]:
X_test = df_test.drop(columns='label')
y_test = df_test['label']
print(X_test.shape, y_test.shape, df_test.shape)

(74, 12) (74,) (74, 13)


In [168]:
from sklearn.preprocessing import OneHotEncoder

encoder = OneHotEncoder(handle_unknown='ignore')
encoder.fit(pd.DataFrame(y_train))
y_train_enc = encoder.transform(pd.DataFrame(y_train)).toarray()
y_test_enc = encoder.transform(pd.DataFrame(y_test)).toarray()

In [169]:
from tensorflow import keras

model = keras.models.Sequential()
model.add(keras.layers.Input(X_train.shape[1]))
model.add(keras.layers.Dropout(0.5))
model.add(keras.layers.Dense(256, activation='relu'))
model.add(keras.layers.Dropout(0.5))
model.add(keras.layers.Dense(256, activation='relu'))
model.add(keras.layers.Dense(4, activation='sigmoid'))

model.compile(optimizer='adam', metrics=['acc'], loss='bce')

In [170]:
model.fit(X_train, y_train_enc,
          validation_data=(X_test, y_test_enc),
          epochs=30)

Epoch 1/30
10/10 [==============================] - 1s 16ms/step - loss: 0.6074 - acc: 0.3401 - val_loss: 0.5354 - val_acc: 0.5405
Epoch 2/30
10/10 [==============================] - 0s 5ms/step - loss: 0.5174 - acc: 0.4932 - val_loss: 0.4844 - val_acc: 0.5270
Epoch 3/30
10/10 [==============================] - 0s 4ms/step - loss: 0.4801 - acc: 0.5510 - val_loss: 0.4573 - val_acc: 0.6757
Epoch 4/30
10/10 [==============================] - 0s 4ms/step - loss: 0.4604 - acc: 0.5748 - val_loss: 0.4385 - val_acc: 0.6757
Epoch 5/30
10/10 [==============================] - 0s 4ms/step - loss: 0.4476 - acc: 0.5408 - val_loss: 0.4251 - val_acc: 0.6757
Epoch 6/30
10/10 [==============================] - 0s 4ms/step - loss: 0.4230 - acc: 0.5748 - val_loss: 0.4125 - val_acc: 0.6622
Epoch 7/30
10/10 [==============================] - 0s 4ms/step - loss: 0.4166 - acc: 0.5850 - val_loss: 0.4094 - val_acc: 0.6622
Epoch 8/30
10/10 [==============================] - 0s 4ms/step - loss: 0.4353 - acc: 0.5

In [171]:
x = model.evaluate(X_test, y_test_enc)

3/3 [==============================] - 0s 1ms/step - loss: 0.4072 - acc: 0.6486


In [172]:
print('Accuracy:', x[1])

Accuracy: 0.6486486196517944


In [72]:
print(type(X_test.toarray()), type(y_test_enc))

AttributeError: 'DataFrame' object has no attribute 'toarray'

In [79]:
X_train

,anger,joy,optimism,sadness,not-hate,hate,non_irony,irony,not-offensive,offensive,...,1,2,3,im_angry,im_disgust,im_fear,im_happy,im_sad,im_surprise,im_neutral
0,0.115456,0.011387,0.195406,0.677751,0.983329,0.016671,0.849077,0.150923,0.757905,0.242095,...,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.966903,0.006743,0.005363,0.020991,0.909270,0.090730,0.921817,0.078183,0.665683,0.334317,...,0.134106,0.529002,0.076143,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.009321,0.957922,0.016562,0.016195,0.972480,0.027520,0.905163,0.094837,0.978436,0.021564,...,0.055007,0.763734,0.035462,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.008275,0.959311,0.019986,0.012427,0.933811,0.066188,0.948829,0.051171,0.909642,0.090358,...,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.976041,0.004920,0.011711,0.007327,0.793186,0.206814,0.432330,0.567670,0.419770,0.580230,...,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
289,0.946807,0.006391,0.020240,0.026562,0.973105,0.026895,0.574206,0.425794,0.783511,0.216489,...,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
290,0.976294,0.006404,0.011243,0.006058,0.855168,0.144832,0.364300,0.635700,0.298030,0.701970,...,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
291,0.941383,0.006008,0.016589,0.036020,0.873737,0.126263,0.574084,0.425916,0.837171,0.162829,...,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
292,0.014331,0.005081,0.015142,0.965446,0.950855,0.049145,0.680607,0.319393,0.834669,0.165331,...,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
